In [63]:
!pip install selenium
import selenium
selenium.__version__

'4.20.0'

In [66]:
!pip install selenium webdriver_manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
service = Service(executable_path = ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')

In [74]:
# 시작 화면 들어가기
import time 
st = 1
from selenium import webdriver
driver = webdriver.Chrome(service = service)
time.sleep(st)
driver.get('https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx/')   # kbo 홈페이지 들어가기
time.sleep(st)

month_button = driver.find_element(By.XPATH, "/html/body/form/div[3]/section/div/div/div[2]/div[3]/div[1]/span[3]/img")   # 달력 클릭
month_button.click()

year_button = driver.find_element(By.XPATH, "/html/body/div/div/div/select[2]")     #연도 아래 화살표
year_button.click()

option_element = driver.find_element(By.XPATH,"/html/body/div/div/div/select[2]/option[13]")  # 2022년 설정
option_element.click()  # 해당 옵션을 클릭하여 선택합니다.

m_button = driver.find_element(By.XPATH, "/html/body/div/div/div/select[1]")   # 달 아래 화살표
m_button.click()

option_element = driver.find_element(By.XPATH,"/html/body/div/div/div/select[1]/option[4]")  # 4월설정
option_element.click()  # 해당 옵션을 클릭하여 선택합니다.

d_button = driver.find_element(By.XPATH, "/html/body/div/table/tbody/tr[1]/td[7]/a")   # 2일 선택
d_button.click()




time.sleep(st)


In [75]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

all_table_data = []

# 288번 반복합니다. 반복횟수 설정
for i in range(168):
    time.sleep(1)

    # 표 데이터를 담을 리스트를 생성합니다.
    table_data = []
    date_info = []

    # 표를 파싱합니다.
    table = driver.find_element(By.XPATH,'/html/body/form/div[3]/section/div/div/div[2]/div[3]/table')
    table_html = table.get_attribute('outerHTML')

    # BeautifulSoup을 사용하여 표를 파싱합니다.
    soup = BeautifulSoup(table_html, 'html.parser')

    # 표의 날짜 정보를 추출합니다.
    date_element = driver.find_element(By.XPATH,'/html/body/form/div[3]/section/div/div/div[2]/div[3]/div[1]/span[2]/span')
    date_info.append(date_element.text)

    # 표의 각 행을 반복하면서 데이터를 추출합니다.
    for row in soup.find_all('tr'):
        row_data = []
        for cell in row.find_all(['td', 'th']):
            row_data.append(cell.get_text().strip())
        table_data.append(row_data)

    # 추출된 데이터를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(table_data[1:], columns=table_data[0])

    # 빈 로우를 추가합니다.
    df.loc[len(df)] = [None] * len(df.columns)

    # 날짜 정보를 데이터프레임에 추가합니다.
    df.insert(0, 'Date', date_info[0])

    # 데이터를 리스트에 추가합니다.
    all_table_data.append(df)

    # 다음 버튼을 클릭합니다.
    next_button = driver.find_element(By.XPATH, "/html/body/form/div[3]/section/div/div/div[2]/div[3]/div[1]/span[4]/input")   # 다음 버튼
    next_button.click()

    time.sleep(1)

# 모든 데이터프레임을 하나로 병합합니다.
final_df = pd.concat(all_table_data, ignore_index=True)

# 모든 데이터를 하나의 엑셀 파일로 저장합니다.
final_df.to_excel('2023년 누적순위.xlsx', index=False)
